# 水果和蔬菜价格数据处理

## 下载压缩文件


### ChatGPT  

::: {.callout-tip}
### 提示词

目的：自动从网页端下载 .zip 数据文件

1. 网址：https://www.ers.usda.gov/data-products/fruit-and-vegetable-prices
2. 网页源文件：view-source:https://www.ers.usda.gov/data-products/fruit-and-vegetable-prices
3. 需要下载的文件：vegetables-####.zip, fruit-###.zip
   - 规则：包含 `vegetables` 或 `fruit` 关键词，且后缀为 `.zip` 的所有文件
   - note: 我通过右击下载链接的方式，发现 **fruit-2013.zip** 对应的下载网址是：`https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/51035/fruit-2013.zip?v=41740`
4. 目标文件夹： 'D:\Github\ds_data\data\Fruit_and_Vegetable_Prices\data_raw'
5. 语言：Python 
   
:::

- 文件清单：
  - `vegetables-2013.zip`, `vegetables-2016.zip`, `vegetables-2020.zip`, 
  - `fruit-2013.zip`, `fruit-2016.zip`, `fruit-2020.zip`

In [9]:
import requests
import re
import os
from urllib.parse import urljoin

# 设置 URL 和目标路径
base_url = "https://www.ers.usda.gov/data-products/fruit-and-vegetable-prices"
target_folder = r"D:\Github\ds_data\data\Fruit_and_Vegetable_Prices\data_raw"

# 创建目标路径
os.makedirs(target_folder, exist_ok=True)

# 获取网页 HTML
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}
response = requests.get(base_url, headers=headers)
html_text = response.text

# 用正则表达式匹配 .zip 链接（fruit 或 vegetables）
pattern = r'href="(.*?(fruit|vegetables)[^"]*?\.zip[^"]*?)"'
matches = re.findall(pattern, html_text, flags=re.IGNORECASE)

# 构造完整链接
zip_links = [urljoin(base_url, m[0]) for m in matches]

print(f"共找到 {len(zip_links)} 个 .zip 文件链接。")

# 下载文件
for link in zip_links:
    filename = os.path.basename(link.split("?")[0])  # 去掉 ?v=xxx 参数
    save_path = os.path.join(target_folder, filename)
    print(f"正在下载：{filename} ...")

    try:
        with requests.get(link, stream=True) as r:
            r.raise_for_status()
            with open(save_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"下载完成：{filename}")
    except Exception as e:
        print(f"下载失败：{filename}，原因：{e}")


共找到 6 个 .zip 文件链接。
正在下载：fruit-2013.zip ...
下载完成：fruit-2013.zip
正在下载：fruit-2016.zip ...
下载完成：fruit-2016.zip
正在下载：fruit-2020.zip ...
下载完成：fruit-2020.zip
正在下载：vegetables-2013.zip ...
下载完成：vegetables-2013.zip
正在下载：vegetables-2016.zip ...
下载完成：vegetables-2016.zip
正在下载：vegetables-2020.zip ...
下载完成：vegetables-2020.zip


In [20]:
import os
os.chdir(target_folder)
print(os.getcwd())
print(os.listdir(target_folder))

D:\Github\ds_data\data\Fruit_and_Vegetable_Prices\data_raw
['copy', 'downloader.log', 'fruit-2013.zip', 'fruit-2016.zip', 'fruit-2020.zip', 'vegetables-2013.zip', 'vegetables-2016.zip', 'vegetables-2020.zip']


### 豆包

> [豆包问答过程](https://www.doubao.com/thread/w24fdaaf190d3f71a)

- 我用豆包只是为了测试我的提示词是否合理，现已达到目标，因此，下面的代码无需重复执行了。

**Round 1** 提示词发送后，豆包给的代码可以运行，但无法下载文件。不过豆包自动生成了一个日志文件，内容如下：

```txt
2025-05-20 00:54:22,003 - INFO - 开始爬取下载链接...
2025-05-20 00:54:24,145 - WARNING - 未找到符合条件的下载链接
```

我把上述内容作为 **Round 2** 的提示词反馈给豆包，它这次返回的代码可以正常下载数据了。 

::: {.callout-tip}
### 提示词

(Round 1)：

目的：自动从网页端下载 .zip 数据文件

1. 网址：https://www.ers.usda.gov/data-products/fruit-and-vegetable-prices
2. 网页源文件：view-source:https://www.ers.usda.gov/data-products/fruit-and-vegetable-prices
3. 需要下载的文件：vegetables-####.zip, fruit-###.zip
   - 规则：包含 `vegetables` 或 `fruit` 关键词，且后缀为 `.zip` 的所有文件
   - note: 我通过右击下载链接的方式，发现 **fruit-2013.zip** 对应的下载网址是：`https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/51035/fruit-2013.zip?v=41740`
4. 目标文件夹： 'D:\Github\ds_data\data\Fruit_and_Vegetable_Prices\data_raw'
5. 语言：Python 

(Round 2)：

2025-05-20 00:54:22,003 - INFO - 开始爬取下载链接...    
2025-05-20 00:54:24,145 - WARNING - 未找到符合条件的下载链接
:::

以下是豆包返回的代码：

In [11]:
# 豆包

import os
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import logging
from datetime import datetime

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='downloader.log'
)
logger = logging.getLogger(__name__)

def get_links(url):
    """提取网页中符合规则的.zip文件链接（包括文本中的隐藏链接）"""
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        logger.error(f"无法访问网页: {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    base_url = urljoin(url, '/')  # 基础URL
    links = []
    
    # 匹配文本中的链接模式（如 Archived 2013 Data Tables for Fruit）
    pattern = r'Archived\s+(\d{4})\s+Data Tables for\s+(Fruit|Vegetables)'
    matches = re.findall(pattern, soup.get_text(), re.IGNORECASE)
    
    for year, category in matches:
        # 构造.zip文件的URL（根据已知格式推断）
        # 示例：https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/51035/fruit-2013.zip
        file_type = category.lower()
        url_suffix = f"{file_type}-{year}.zip"
        # 注意：此处URL路径可能需要根据实际情况调整，当前为示例路径
        zip_url = f"https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/51035/{url_suffix}"
        
        # 验证链接格式（包含fruit/vegetables和.zip）
        if (file_type in ['fruit', 'vegetables'] and 
            zip_url.lower().endswith('.zip')):
            links.append(zip_url)
    
    return links

def download_file(url, target_folder):
    # 保持原有逻辑不变（见上文）
    os.makedirs(target_folder, exist_ok=True)
    filename = os.path.basename(urlparse(url).path)
    local_path = os.path.join(target_folder, filename)
    
    if os.path.exists(local_path):
        logger.info(f"文件已存在: {filename}")
        return local_path
    
    try:
        logger.info(f"开始下载: {filename}")
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        logger.info(f"下载完成: {filename}")
        return local_path
    except Exception as e:
        logger.error(f"下载失败: {filename}, 错误: {e}")
        return None

if __name__ == "__main__":
    url = "https://www.ers.usda.gov/data-products/fruit-and-vegetable-prices"
    target_folder = r"D:\Github\ds_data\data\Fruit_and_Vegetable_Prices\data_raw"
    
    logger.info("开始爬取下载链接...")
    links = get_links(url)
    
    if not links:
        logger.warning("未找到符合条件的下载链接")
    else:
        logger.info(f"找到 {len(links)} 个符合条件的下载链接: {links}")
        for link in links:
            download_file(link, target_folder)
    
    logger.info("下载任务完成")

## 解压缩包-合并单个 csv 文件

::: {.callout-tip}
### 提示词

- 把以 `fruit` 开头的 .zip 文件解压到 [fruit] 文件夹中
- 把以 `vegetables` 开头的 zip 文件解压到 [vegetables] 文件夹中
- 完成后，统计上述两个新生成的子文件夹内的文件数量
  
:::

In [21]:
import zipfile

# 创建目标子文件夹
fruit_folder = os.path.join(target_folder, "fruit")
vegetables_folder = os.path.join(target_folder, "vegetables")
os.makedirs(fruit_folder, exist_ok=True)
os.makedirs(vegetables_folder, exist_ok=True)

# 遍历 target_folder 下的所有 zip 文件并解压
for filename in os.listdir(target_folder):
    if filename.lower().endswith(".zip"):
        zip_path = os.path.join(target_folder, filename)
        if filename.startswith("fruit"):
            extract_path = fruit_folder
        elif filename.startswith("vegetables"):
            extract_path = vegetables_folder
        else:
            continue
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

# 统计文件数量
fruit_files = [f for f in os.listdir(fruit_folder) if os.path.isfile(os.path.join(fruit_folder, f))]
vegetables_files = [f for f in os.listdir(vegetables_folder) if os.path.isfile(os.path.join(vegetables_folder, f))]

print(f"fruit 文件夹内文件数量: {len(fruit_files)}")
print(f"vegetables 文件夹内文件数量: {len(vegetables_files)}")

fruit 文件夹内文件数量: 117
vegetables 文件夹内文件数量: 177


### 合并与结构化

#### 观察各个年度的文件命名规则的差别

[fruit] 文件夹下的前 8 个文件列举如下：
  
   ![](https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/20250520012922.png)

仔细对比后发现：

- 2016 年数据的有两个文件：一个是 .csv 格式的，另一个是 .xlsx 格式的。对比文档内容后发现，两份数据文件的内容完全相同。进一步抽查了其十种水果在 2016 年的数据，发现都存在上述规律。
  - 最终决定：删除 [fruit] 文件夹下的 .csv 文件，保留 .xlsx 文件，以便后续采用循环语句进行统一处理。 
- 为了保持各年度上的文件命名方式统一，做如下处理：
  - 将 [fruit] 文件夹下的所有文件名中的 ` ` (空格) 替换为 `_` (下划线)；
  - 将文件名中的有英文单词替换为小写字母。

#### 结构化处理

- 分析 csv 文件的内部结构：非结构化 >> 结构化
   
   ![](https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/20250520013804.png)

   ![](https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/20250520013503.png) 

   ![](https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/20250520013356.png)

   ![](https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/20250520092820.png)

   ![](https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/20250520091552.png)

   ![](https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/20250520092543.png)

Apricots 的数据结构如下：

![](https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/20250520084449.png)

我们需要把上述半结构化的数据转换成整洁数据。浏览网页，发现 [ALL FRUITS – Average prices (CSV format)](https://ers.usda.gov/sites/default/files/_laserfiche/DataFiles/51035/Fruit-Prices-2022.csv?v=74889) 页面已经有了整洁数据的格式。该数据文件的格式为：

 ![](https://fig-lianxh.oss-cn-shenzhen.aliyuncs.com/20250520083653.png)

- 观察到 csv 文件的第一行是表头，第二行是数据类型，第三行是数据内容
- 因此，读取 csv 文件时，指定 `header=2`，并且 `skiprows=3`，这样就可以直接读取数据了
- 读取数据时，指定 `usecols` 参数，选择需要的列